<a href="https://colab.research.google.com/github/polinauni/IntroToCL/blob/main/CL_Final_project_Bogdanova_Crosslingual_paraphrase_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project

First, we load all the needed libraries for the project:



In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install bertviz transformers
!pip install accelerate --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 4.2 MB/s eta 0:00:00
   ━━━

In [4]:
# Mounting Googledrive to store hidden embedding layers

from google.colab import drive
drive.mount('/content/drive/MyDrive/finalproject')

ValueError: Mountpoint must be in a directory that exists

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base")
print(tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaTokenizerFast(name_or_path='FacebookAI/xlm-roberta-base', vocab_size=250002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)


In [6]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding

ds_de = load_dataset("google-research-datasets/paws-x", "de")
ds_en = load_dataset("google-research-datasets/paws-x", "en")

print(ds_de)
print(ds_en)

README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.91M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/352k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/345k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/307k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})


In [7]:
# In order to do multilungual and initially cross-linguag (on the test) paraphrase detection
# It is necessary to cancatenate English and German datasets

from datasets import concatenate_datasets

# Concatination step just stack datasets on top of each other
# 1. Concatenate the 'train' splits from English & German
train_combined = concatenate_datasets([ds_en['train'], ds_de['train']])

# 2. Concatenate the 'validation' splits
validation_combined = concatenate_datasets([ds_en['validation'], ds_de['validation']])

# 3. Concatenate the 'test' splits
test_combined = concatenate_datasets([ds_en['test'], ds_de['test']])

# 4. Shuffle each split so that examples from EN & DE are mixed
train_combined = train_combined.shuffle(seed=42)
validation_combined = validation_combined.shuffle(seed=42)
test_combined = test_combined.shuffle(seed=42)

# 5. Store everything in a single DatasetDict
ds_combined = DatasetDict({
    'train': train_combined,
    'validation': validation_combined,
    'test': test_combined
})

print(ds_combined)

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 98802
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 4000
    })
})


In [8]:
# Subset the dataset to ~500 training examples, 200 for validation/test

ds_combined['train'] = ds_combined['train'].shuffle(seed=42).select(range(500))
ds_combined['validation'] = ds_combined['validation'].shuffle(seed=42).select(range(200))
ds_combined['test'] = ds_combined['test'].shuffle(seed=42).select(range(200))

#def truncate(example):
#    return {
#       'text': " ".join(example['text'].split()[:50]),
#        'label': example['label']
#    }
#
# Take 128 random examples for train and 32 validation
#small_imdb_dataset = DatasetDict(
#    train=imdb_dataset['train'].shuffle(seed=24).select(range(128)).map(truncate),
#    val=imdb_dataset['train'].shuffle(seed=24).select(range(128, 160)).map(truncate),
#)

In [ ]:
# Print out the first 10 examples from the train dataset to verify correctness
#small_imdb_dataset["train"][:10]

In [9]:
# We need to load the data for use in Pytorch

def tokenize_function(examples):
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        padding='max_length',
        truncation=True,
        max_length=128 # Token leght for truncation and handling too long sentences
    )

small_tokenized_dataset = ds_combined.map(tokenize_function, batched=True)

# remove the original text columns ITS OPTIONAL, CAN KEEP
#small_tokenized_dataset = small_tokenized_dataset.remove_columns(["sentence1", "sentence2"])

# rename 'label' to 'labels' > better for BERT like models that expect this namimg
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")

# set format for PyTorch
small_tokenized_dataset.set_format("torch")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
# Check what the first two sequences look like

small_tokenized_dataset['train'][0:2]

{'id': tensor([33632, 43589]),
 'sentence1': ['Er wurde am 21. Mai 1897 in Sofia geboren und starb am 15. Juni 1945 in Jambol.',
  'Miloslav Mečíř gewann gegen John McEnroe mit 6: 0, 3 - 6, 6 - 2, 6 - 2 im Finale.'],
 'sentence2': ['Er wurde am 21. Mai 1897 in Yambol geboren und starb am 15. Juni 1945 in Sofia.',
  'Miloslav Mečíř gewann im Finale 6 - 0, 3 - 6, 6 - 2, 6 - 2 gegen John McEnroe.'],
 'labels': tensor([0, 1]),
 'input_ids': tensor([[     0,   1004,   4180,    444,   7245,   3596, 117430,     23,  80748,
           54997,    165,   6057,    275,    444,   6281,  17223,  18608,     23,
             823,  36846,      5,      2,      2,   1004,   4180,    444,   7245,
            3596, 117430,     23,   2369,  36846,  54997,    165,   6057,    275,
             444,   6281,  17223,  18608,     23,  80748,      5,      2,      1,
               1,      1,      1,      1,      1,      1,      1,      1,      1,
               1,      1,      1,      1,      1,      1,      1,   

In [ ]:
# NO NEED AS IT HANDLED AUTOMALICALLY IN THE TRAINER CLASS
# Create a Dataloader for training and test datasets to iterate over batches of data

#from torch.utils.data import DataLoader

#train_dataloader = DataLoader(small_tokenized_dataset['train'], batch_size=16)
#eval_dataloader = DataLoader(small_tokenized_dataset['val'], batch_size=16)

In [ ]:
# NO NEED TO DO IT ONE MORE TIME ALREADY DONE
# Load the dataset again and tokenize it

# from datasets import load_dataset, DatasetDict
# from transformers import DataCollatorWithPadding

# imdb_dataset = load_dataset("imdb")
# # we had loaded the imdb dataset already above - if not, outcomment this line
# # Make sure you have the right tokenizer
# tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")


# # Just take the first 50 tokens for speed on CPU
# def truncate(example):
#     return {
#         'text': " ".join(example['text'].split()[:200]),
#         'label': example['label']
#     }

# # Take 128 random examples for train and 32 validation
# small_imdb_dataset = DatasetDict(
#     train=imdb_dataset['train'].shuffle(seed=24).select(range(128)).map(truncate),
#     val=imdb_dataset['train'].shuffle(seed=24).select(range(128, 160)).map(truncate),
# )

# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding=True, truncation=True)

# small_tokenized_dataset = small_imdb_dataset.map(tokenize_function, batched=True, batch_size=16)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
# Specifyng all the trainig parameters
from transformers import DataCollatorWithPadding
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)
accuracy = evaluate.load("accuracy")


arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/final_project/paraphrase_detection_1st",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=8,
    num_train_epochs=5,
    eval_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['validation'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-13-d1a3d55a4cf2>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
# Model is trained

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.700400,0.684799,0.575000
2,0.696600,0.685672,0.575000
3,0.697300,0.679097,0.575000
4,0.658000,0.708531,0.520000
5,0.625500,0.687184,0.560000


TrainOutput(global_step=160, training_loss=0.6701039433479309, metrics={'train_runtime': 278.9371, 'train_samples_per_second': 8.963, 'train_steps_per_second': 0.574, 'total_flos': 164444409600000.0, 'train_loss': 0.6701039433479309, 'epoch': 5.0})

In [21]:
# Check for the label distribution in the train data set
from collections import Counter
Counter(small_tokenized_dataset["train"]["labels"])

Counter({tensor(0): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(0): 1,
         tensor(0): 1,
         tensor(0): 1,
         tensor(1): 1,
         tensor(0): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(0): 1,
         tensor(1): 1,
         tensor(0): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(0): 1,
         tensor(0): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(0): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(0): 1,
         tensor(1): 1,
         tensor(0): 1,
         tensor(1): 1,
         tensor(0): 1,
         tensor(1): 1,
         tensor(0): 1,
         tensor(1): 1,
         tensor(1): 1,
         tensor(0): 1,
         tensor(0): 1,
         tensor(1): 1,
         tensor(0): 1,
         te

# 2d Training

In [24]:
# Second training run

train_combined = concatenate_datasets([ds_en['train'], ds_de['train']])

# 2. Concatenate the 'validation' splits
validation_combined = concatenate_datasets([ds_en['validation'], ds_de['validation']])

# 3. Concatenate the 'test' splits
test_combined = concatenate_datasets([ds_en['test'], ds_de['test']])

# 4. Shuffle each split so that examples from EN & DE are mixed
train_combined = train_combined.shuffle(seed=42)
validation_combined = validation_combined.shuffle(seed=42)
test_combined = test_combined.shuffle(seed=42)

# 5. Store everything in a single DatasetDict
ds_combined = DatasetDict({
    'train': train_combined,
    'validation': validation_combined,
    'test': test_combined
})

print(ds_combined)


ds_combined['train'] = ds_combined['train'].shuffle(seed=42).select(range(2000))
ds_combined['validation'] = ds_combined['validation'].shuffle(seed=42).select(range(500))
ds_combined['test'] = ds_combined['test'].shuffle(seed=42).select(range(500))

def tokenize_function(examples):
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        padding='max_length',
        truncation=True,
        max_length=150 # Token leght for truncation and handling too long sentences
    )

small_tokenized_dataset = ds_combined.map(tokenize_function, batched=True)

# remove the original text columns ITS OPTIONAL, CAN KEEP
#small_tokenized_dataset = small_tokenized_dataset.remove_columns(["sentence1", "sentence2"])

# rename 'label' to 'labels' > better for BERT like models that expect this namimg
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")

# set format for PyTorch
small_tokenized_dataset.set_format("torch")

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 98802
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 4000
    })
})


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [25]:
print("Train size:", ds_combined["train"].num_rows)
print("Validation size:", ds_combined["validation"].num_rows)
print("Test size:", ds_combined["test"].num_rows)

Train size: 2000
Validation size: 500
Test size: 500


In [26]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)
accuracy = evaluate.load("accuracy")


arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/final_project/paraphrase_detection_2d",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=8,
    num_train_epochs=10,
    eval_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=1e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['validation'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-26-fed2ef8d9c9a>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.682600,0.689415,0.602000
2,0.669500,0.679147,0.600000
3,0.668600,0.679363,0.600000
4,0.597100,0.803623,0.498000
5,0.571300,0.746013,0.600000
6,0.526500,0.825123,0.610000
7,0.382000,0.803414,0.660000
8,0.395200,0.780931,0.698000
9,0.251400,0.793717,0.700000
10,0.292300,0.841879,0.702000


TrainOutput(global_step=1250, training_loss=0.5293026403427123, metrics={'train_runtime': 1547.4478, 'train_samples_per_second': 12.925, 'train_steps_per_second': 0.808, 'total_flos': 1541666340000000.0, 'train_loss': 0.5293026403427123, 'epoch': 10.0})

In [28]:
# Model evaluation

results = trainer.predict(small_tokenized_dataset['validation'])
print("Validation results:", results.metrics)

Validation results: {'test_loss': 0.6791465282440186, 'test_accuracy': 0.6, 'test_runtime': 4.1857, 'test_samples_per_second': 119.455, 'test_steps_per_second': 7.645}


# 3d Training

In [29]:
train_combined = concatenate_datasets([ds_en['train'], ds_de['train']])

# 2. Concatenate the 'validation' splits
validation_combined = concatenate_datasets([ds_en['validation'], ds_de['validation']])

# 3. Concatenate the 'test' splits
test_combined = concatenate_datasets([ds_en['test'], ds_de['test']])

# 4. Shuffle each split so that examples from EN & DE are mixed
train_combined = train_combined.shuffle(seed=42)
validation_combined = validation_combined.shuffle(seed=42)
test_combined = test_combined.shuffle(seed=42)

# 5. Store everything in a single DatasetDict
ds_combined = DatasetDict({
    'train': train_combined,
    'validation': validation_combined,
    'test': test_combined
})

print(ds_combined)


ds_combined['train'] = ds_combined['train'].shuffle(seed=42).select(range(2500))
ds_combined['validation'] = ds_combined['validation'].shuffle(seed=42).select(range(500))
ds_combined['test'] = ds_combined['test'].shuffle(seed=42).select(range(500))

def tokenize_function(examples):
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        padding='max_length',
        truncation=True,
        max_length=150 # Token leght for truncation and handling too long sentences
    )

small_tokenized_dataset = ds_combined.map(tokenize_function, batched=True)

# remove the original text columns ITS OPTIONAL, CAN KEEP
#small_tokenized_dataset = small_tokenized_dataset.remove_columns(["sentence1", "sentence2"])

# rename 'label' to 'labels' > better for BERT like models that expect this namimg
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")

# set format for PyTorch
small_tokenized_dataset.set_format("torch")

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 98802
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 4000
    })
})


Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [31]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)
accuracy = evaluate.load("accuracy")


arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/final_project/paraphrase_detection_3d",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=8,
    num_train_epochs=8,
    eval_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=3e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)

from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,  # Stop after 'patience' epochs with no improvement
    early_stopping_threshold=0.0  # How much improvement must be seen
)


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['validation'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[early_stopping],
    compute_metrics=compute_metrics
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-31-556566052387>:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [32]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.695000,0.678922,0.600000
2,0.688600,0.676200,0.600000
3,0.689800,0.680287,0.600000
4,0.684400,0.676174,0.600000
5,0.689500,0.680006,0.600000
6,0.692500,0.679816,0.600000


KeyboardInterrupt: 

The fine-tuning above was interrupted due to the no change in the accuracy.

In [33]:
results = trainer.predict(small_tokenized_dataset["test"])
print("Test results:", results.metrics)

Epoch,Training Loss,Validation Loss,Accuracy
1,0.695000,0.678922,0.600000
2,0.688600,0.676200,0.600000
3,0.689800,0.680287,0.600000
4,0.684400,0.676174,0.600000
5,0.689500,0.680006,0.600000
6,0.692500,0.679816,0.600000


Test results: {'test_loss': 0.685560941696167, 'test_accuracy': 0.566, 'test_runtime': 4.5015, 'test_samples_per_second': 111.075, 'test_steps_per_second': 3.554}


In [ ]:
# test_str = "I love this movie!"

# fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(
#     "/content/drive/MyDrive/final_project/paraphrase_detection_1st/checkpoint-40")
# model_inputs = tokenizer(test_str, return_tensors="pt")
# prediction_id = torch.argmax(fine_tuned_model(**model_inputs).logits, dim=-1).item()

# label_names = ["NOT_PARAPHRASE", "PARAPHRASE"]
# print(f"Prediction: {label_names[prediction_id]}")

In [38]:
import torch
# 1) Load your fine-tuned model from a saved checkpoint.
#    Make sure the 'num_labels=2' matches your fine-tuning setup.
checkpoint_path = "/content/drive/MyDrive/final_project/paraphrase_detection_2d/checkpoint-1250"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)
model.eval()  # set model to evaluation mode

# 2) Load the tokenizer used during training (e.g., xlm-roberta-base).
tokenizer_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# 3) Define label mapping.
#    We'll assume label "0" => NOT_PARAPHRASE ("NO"), "1" => PARAPHRASE ("YES").
label_names = ["NO", "YES"]

# 4) Prepare some cross-lingual sentence pairs (English + German/other).
#    We'll specify which pairs we consider paraphrases or not, just for illustration.

test_pairs = [
    # (sentence1, sentence2, "Expected label")
    ("I love cats", "Cats are my favourite animal", "YES"),  # paraphrase
    ("Where is the library?", "Where do I find the library?", "YES"),  # paraphrase
    ("He found a new job", "He lose his job", "NO"),   # not paraphrase
]

# 5) Run inference on each pair.
for idx, (sent1, sent2, expected) in enumerate(test_pairs):
    # Encode both sentences together
    model_inputs = tokenizer(sent1, sent2, return_tensors="pt", truncation=True)

    # Forward pass
    with torch.no_grad():
        outputs = model(**model_inputs)
        logits = outputs.logits

    # Prediction
    predicted_label_id = torch.argmax(logits, dim=-1).item()
    predicted_label_name = label_names[predicted_label_id]

    print(f"Example {idx+1}:")
    print(f"  Sentence 1: {sent1}")
    print(f"  Sentence 2: {sent2}")
    print(f"  Predicted label: {predicted_label_name}  (Expected: {expected})\n")

Example 1:
  Sentence 1: I love cats
  Sentence 2: Cats are my favourite animal
  Predicted label: NO  (Expected: YES)

Example 2:
  Sentence 1: Where is the library?
  Sentence 2: Where do I find the library?
  Predicted label: YES  (Expected: YES)

Example 3:
  Sentence 1: He found a new job
  Sentence 2: He lose his job
  Predicted label: NO  (Expected: NO)



In [ ]:
# Model loading for specific check points

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/final_project/paraphrase_detection_1st/checkpoint-8"
)
model_inputs = tokenizer(
    ["Example sentence 1", "Example sentence 2"],  # or something from your dataset
    ["Das ist ein Beispiel", "Dies ist ein Test"], # for cross-lingual pairs
    padding=True, truncation=True, return_tensors="pt"
)
outputs = fine_tuned_model(**model_inputs, output_hidden_states=True)
all_hidden_states = outputs.hidden_states

In [ ]:
# Store hidden states for each layer for the checkpoint in a folder

import torch
import os

path = "/content/drive/MyDrive/results_vis"
#path = "results_vis"

layer=0
if not os.path.exists(path):
  os.mkdir(path)

all_hidden_states = outputs["hidden_states"]
num_layers = len(all_hidden_states)

texts = some_dataset["sentence1"]
labels = some_dataset["labels"]


for layer_idx in range(num_layers):
  layer_path = os.path.join(path, f"layer_{layer_idx}")
  if not os.path.exists(layer_dir):
    os.mkdir(layer_dir)

  cls_embeddings = []
  metadata = []


  # Suppose we have N examples in the batch
  batch_size = all_hidden_states[layer_idx].shape[0]
    for ex_idx in range(batch_size):
        # [CLS] embedding for this layer
        cls_vector = all_hidden_states[layer_idx][ex_idx][0]  # shape [hidden_dim]
        cls_embeddings.append(cls_vector)

        # Prepare metadata (text + label)
        # E.g. store the first sentence or both if you want
        # If you removed them from the dataset, you'll need to have them saved somewhere else.
        text_str = texts[ex_idx]
        label_str = str(labels[ex_idx])
        metadata.append([text_str, label_str])

    # Turn list of embeddings into a Tensor
    cls_tensor = torch.stack(cls_embeddings)  # shape [batch_size, hidden_dim]

    # Write to TensorBoard
    writer = SummaryWriter(layer_dir)
    writer.add_embedding(
        cls_tensor,
        metadata=metadata,
        metadata_header=["Text", "Label"]
    )
    writer.close()

  # while example in range(len(outputs['hidden_states'][layer])):
  #   sp_token_position = 0
  #   for token in model_inputs['input_ids'][example]:
  #     if token != 101:
  #       sp_token_position += 1
  #     else:
  #       tensor = outputs['hidden_states'][layer][example][sp_token_position]
  #       tensors.append(tensor)
  #       break

  #   label = [
  #     small_tokenized_dataset['val']['text'][example],
  #     str(small_tokenized_dataset['val']['labels'][example])
  #   ]
  #   labels.append(label)
  #   example +=1

  # writer=SummaryWriter(path+'/layer_' + str(layer))
  # writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['Text','Emotion'])

  # layer+=1